In [1]:
import import_ipynb
import IntimeLibrary as il
import FlieLibrary as fl
import pandas as pd
import numpy as np
import pickle

importing Jupyter notebook from IntimeLibrary.ipynb
importing Jupyter notebook from FlieLibrary.ipynb


c:\users\kss09\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [2]:
#None값 열 제거
fl.depart_data.dropna(axis=1, inplace=True)

In [3]:
fl.depart_data.drop(["날짜", "time", "TM", "현황"],axis=1, inplace=True)
fl.depart_data.drop(["VIS", "PS"],axis=1, inplace=True)

In [4]:
#one-hot-encoding
fl.depart_data = pd.get_dummies(fl.depart_data, columns=['항공사',"ORIGIN","DEST", "weekend"])

In [5]:
fl.depart_data['Date']=fl.depart_data['Date'].astype(np.int64)
fl.depart_data['Time']=fl.depart_data['Time'].astype(np.int64)
fl.depart_data["DelayRate"]=fl.depart_data["DelayRate"].astype(np.int64)

In [6]:
#테스트 세트 생성
def split_flight_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    flight_indices = shuffled_indices[test_set_size:]
    return data.iloc[flight_indices], data.iloc[test_indices]

In [7]:
# 무작위로 샘플 선택하여 데이터셋의 20% 분리
flight_set, test_set = split_flight_test(fl.depart_data, 0.2)
print(len(flight_set), "flight +", len(test_set), "test")

108924 flight + 27231 test


In [8]:
fl.depart_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136155 entries, 0 to 136154
Data columns (total 28 columns):
CA_TOT         136155 non-null int64
Date           136155 non-null int64
DelayRate      136155 non-null int64
PA             136155 non-null int64
TD             136155 non-null int64
TMP            136155 non-null int64
Time           136155 non-null int64
WD             136155 non-null int64
WSPD           136155 non-null int64
항공사_대한항공       136155 non-null uint8
항공사_아시아나항공     136155 non-null uint8
항공사_에어부산       136155 non-null uint8
항공사_에어필립       136155 non-null uint8
항공사_이스타항공      136155 non-null uint8
항공사_제주항공       136155 non-null uint8
항공사_진에어        136155 non-null uint8
항공사_티웨이항공      136155 non-null uint8
ORIGIN_김포공항    136155 non-null uint8
ORIGIN_제주공항    136155 non-null uint8
DEST_김포공항      136155 non-null uint8
DEST_제주공항      136155 non-null uint8
weekend_FRI    136155 non-null uint8
weekend_MON    136155 non-null uint8
weekend_SAT    136155 non-null uint8
w

In [19]:
from sklearn.model_selection import train_test_split
flight_set, test_set = train_test_split(fl.depart_data, test_size=0.35, random_state=42)

In [20]:
#계층별로 데이터셋에 충분한 샘플 수가 있어야함 -> 중요도 편향 방지
fl.depart_data["delay"] = fl.depart_data['DelayRate']
fl.depart_data["delay"].where(fl.depart_data["DelayRate"] < 5, 5.0, inplace=True)

In [29]:
#계층별 샘플링
from sklearn.model_selection import StratifiedShuffleSplit
 
split = StratifiedShuffleSplit(n_splits=1, test_size=0.35, random_state=42)
for flight_index, test_index in split.split(fl.depart_data, fl.depart_data["delay"]):
    strat_flight_set = fl.depart_data.loc[flight_index]
    strat_test_set = fl.depart_data.loc[test_index]

In [30]:
#전체 데이터셋에서 delay카테고리의 비율측정
fl.depart_data["delay"].value_counts() / len(fl.depart_data)

0    0.198134
1    0.194022
2    0.170695
4    0.148706
3    0.146377
5    0.142066
Name: delay, dtype: float64

In [31]:
#delay특성 삭제 데이터 원래 상태로 복구
for set_ in (strat_flight_set, strat_test_set):
    set_.drop("delay", axis=1, inplace=True)

In [32]:
#훈련데이터셋 손상 방지 복사본 만들어 사용
data = strat_flight_set.copy()

In [33]:
data = strat_flight_set.drop("DelayRate", axis=1)
data_labels = strat_flight_set["DelayRate"].copy()

In [34]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(data, data_labels)

c:\users\kss09\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [35]:
saved_model = pickle.dumps(forest_reg)

In [36]:
from sklearn.externals import joblib

joblib.dump(forest_reg, 'intime.pkl')

['intime.pkl']